# Fish Welfare Project
## Part 2: Fish Count Database

* Author: Angelina Li
* Date: 2019/10/09
* Description: This notebook attempts to clean data from the fish count database.

## Notebook tasks
1. Import in fish count data.
2. Clean names in the fish count dataset.
3. Clean and repair data on the total weight, average weight, and average number of species per year.
4. Save data.

In [1]:
import os
import pandas as pd
import re
import random

In [2]:
MAIN_DIR = ".."
DATA_DIR = os.path.join(MAIN_DIR, "data")
COUNT_INPUT_DIR = os.path.join(DATA_DIR, "fish_count", "input")
COUNT_OUTPUT_DIR = os.path.join(DATA_DIR, "fish_count", "output")

**1. Import the fish count databases**

In [3]:
CT_DECAPOD_FP = os.path.join(COUNT_INPUT_DIR, "Farmed-decapods-2015.xlsx")
CT_FISH_FP = os.path.join(COUNT_INPUT_DIR, "Farmed-fishes-2015.xlsx")
CT_WILD_FP = os.path.join(COUNT_INPUT_DIR, "fishcount_estimated_wild_fish_2007-2016.xlsx")

In [4]:
# grab all of the datas
deca_df = pd.read_excel(CT_DECAPOD_FP, sheet_name="Decapods", header=8)
print(len(deca_df))
deca_df.head(3)

1852


,Country,FAO Species Category,Scientific name,Decapod species?,Crustacean species?,Class,Order,Family,Multi-species?,Year,Production (t),EMW id,Estimated mean weight (lower),Estimated mean weight (upper),mean weight (lower),mean weight (upprr),Numbers (lower) millions,Numbers (upper) millions
0,Afghanistan,Cyprinids nei,Cyprinidae,N,N,Actinopterygii,CYPRINIFORMES,Cyprinidae,NaN,2015.0,1000.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,Afghanistan,Rainbow trout,Oncorhynchus mykiss,N,N,Actinopterygii,SALMONIFORMES,Salmonidae,NaN,2015.0,150.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,Albania,Bighead carp,Hypophthalmichthys nobilis,N,N,Actinopterygii,CYPRINIFORMES,Cyprinidae,NaN,2015.0,16.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [5]:
fish_df = pd.read_excel(CT_FISH_FP, sheet_name="Fish species", header=6)
print(len(fish_df))
fish_df.head(3)

1853


,Country,FAO Species Category,Scientific name,Fish species?,Class,Order,Family,Multi-species?,Year,Production (t),EMW id,Estimated mean weight (lower),Estimated mean weight (upper),mean weight (lower),mean weight (upper),Numbers (lower) millions,Numbers (upper) millions
0,Afghanistan,Rainbow trout,Oncorhynchus mykiss,Y,Actinopterygii,SALMONIFORMES,Salmonidae,N,2015.0,150.0,155.0,210.0,5000.0,210.000000,5000.000000,0.030000,0.714286
1,Afghanistan,Cyprinids nei,Cyprinidae,Y,Actinopterygii,CYPRINIFORMES,Cyprinidae,NaN,2015.0,1000.0,NaN,0.0,0.0,322.064283,1081.212063,0.924888,3.104970
2,Albania,Bighead carp,Hypophthalmichthys nobilis,Y,Actinopterygii,CYPRINIFORMES,Cyprinidae,N,2015.0,16.0,29.0,500.0,1500.0,500.000000,1500.000000,0.010667,0.032000


In [6]:
wild_df = pd.read_excel(CT_WILD_FP, sheet_name="Sheet1", header=17)
print(len(wild_df))
wild_df.head(3)

12045


,Country,FAO Species Category,Scientific name,Fish species?,Class,Multi-species?,Year,Production (t),EMW id,Estimated mean weight EMW (lower) g,Estimated mean weight EMW (upper) g,Global Generic estimated mean weight for class GEMW (lower) g,Global Generic estimated mean weight for class GEMW (upper) g,Mean weight used (lower) g,Mean weight used (upper) g,Estimated numbers (lower) millions,Estimated numbers (upper) millions
0,Afghanistan,Freshwater fishes nei,NaN,Y,Includes species from > 1 class,NaN,2007-2016,1000.0,NaN,0,0,37.8921,96.5228,37.8921,96.5228,10.360251,26.390735
1,Albania,"Angelsharks, sand devils nei",Squatinidae,Y,Elasmobranchii (sharks and rays),Y,2007-2016,16.0,23,1683.72,19793.8,5950.39,10539.4,1683.72,19793.8,0.000808,0.009503
2,Albania,Atlantic bluefin tuna,Thunnus thynnus,Y,Actinopterygii (ray-finned fishes),N,2007-2016,18.0,51,262000,262000,37.7549,96.1746,262000,262000,0.000069,0.000069


**2. Clean names**

In [7]:
random.sample(set(wild_df["FAO Species Category"].to_list()), 500)

['Weathervane scallop',
 'Longfin yellowtail',
 'Sea chubs nei',
 'Torpedo rays',
 'Ruffs, barrelfishes nei',
 'Oreo dories nei',
 'Starry smooth-hound',
 'Great lanternshark',
 'Pompanos nei',
 'Seaweeds nei',
 'Hairtails, scabbardfishes nei',
 'North Pacific hake',
 'Masu(=Cherry) salmon',
 'Shortnose greeneye',
 'Lake cisco',
 'Narrow-barred Spanish mackerel',
 'Pacific cornetfish',
 'Aurora rockfish',
 53,
 'Cardinal fishes nei',
 'Copper shark',
 "Skottsberg's gigartina",
 'Smooth oreo dory',
 81,
 'Smooth hammerhead',
 'Loweye catfishes nei',
 44,
 'European flat oyster',
 'Yellowtail amberjack',
 'Dungat grouper',
 'Parrotfish',
 'Rudd',
 332,
 323,
 'West African ilisha',
 'Gars nei',
 'Cholga mussel',
 'Antarctic flying squid',
 'African striped grunt',
 'Milk shark',
 'Sevan trout',
 'White sturgeon',
 'Sword razor shell',
 'Largemouth black bass',
 'Characins nei',
 'Southern hake',
 'Argentinian sandperch',
 'Blackfin snapper',
 'Blacktail comber',
 'Pacific ocean perch',
 

Cleaning ideas (for each idea, generate a new column testing that idea):
* Make everything a string.
* Remove the suffix "nei"?
* Make a new column per replacement that is needed, including a final column removing the pattern "(=\[A-Za-z\])" altogether.
* Lower the entire string, replace all punctuation with spaces, and standardize spaces.

Let's write a series of small functions to clean each name given some assumptions, and then we'll tackle applying it to a dataset.

In [ ]:
def eng_name_standardized(nm):
    converted = re.sub("[^A-Za-z ]", " ", nm)
    stripped = re.sub(" +", " ", nm)
    return stripped.lower()
    
def eng_name_remove_nei(nm):
    return re.sub(" nei$", "", nm)

def eng_name_get_copies(nm):
    # convert names like this 'Chinook(=Spring=King) salmon' to a list of names.
    variation_regex = "\((=[A-Za-z]+)+\)"
    matches = re.search(variation_regex, nm)
    if not re.search(variation_regex, nm):
        return eng_name_standardized(nm)
    strip_parens = lambda name: re.sub(variation_regex, "", name)
    
    split_nm = re.split("([\(\)=])", nm)
    
    clean_lines = []
    replacing = False
    string_template = ""
    
    for i, segment in enumerate(split_nm):
        if not segment:
            continue
        if segment == "(":
            replacing = True
            assert i > 0
            # if this is the end of the string
            if i == len(split_nm) - 1 and nm.count("(") == 1:
                string_template = "{}"
            last_segment = split_nm[i-1]
            # replace the last word with a 
            replaced_seg = re.sub("[A-Za-z]+$", "{}", last_segment)
            cleaned_list = split_nm.copy()
            cleaned_list[i-1] = replaced_seg
            cleaned_string = strip_parens("".join(cleaned_list))
            print('"' + cleaned_string + '"')
        if segment == "=":
            pass
        if segment == ")":
            replacing = False

test_string = "Chinook(=Spring=King) salmon"
eng_name_get_copies(test_string)